In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pythainlp import word_tokenize
from pythainlp.ulmfit import process_thai


###### Sentiment Analysis & Create Model ######

In [2]:
with open("wisesight-sentiment-master/kaggle-competition/train.txt", encoding="utf8") as f:
    texts = [line.strip() for line in f.readlines()]

with open("wisesight-sentiment-master/kaggle-competition/train_label.txt", encoding="utf8") as f:
    categories = [line.strip() for line in f.readlines()]

all_df = pd.DataFrame({"category":categories, "texts":texts})
all_df.to_csv('all_df.csv',index=False)
all_df.shape

(24063, 2)

In [3]:
all_df["processed"] = all_df.texts.map(lambda x: "|".join(process_thai(x)))
all_df["wc"] = all_df.processed.map(lambda x: len(x.split("|")))
all_df["uwc"] = all_df.processed.map(lambda x: len(set(x.split("|"))))

In [4]:
#when finding hyperparameters
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(all_df, test_size=0.15, random_state=1412)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)

In [5]:
#dependent variables
y_train = train_df["category"]
y_valid = valid_df["category"]

In [6]:
#text faetures
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(tokenizer=process_thai, ngram_range=(1,2), min_df=20, sublinear_tf=True)
tfidf_fit = tfidf.fit(all_df["texts"])
text_train = tfidf_fit.transform(train_df["texts"])
text_valid = tfidf_fit.transform(valid_df["texts"])
text_train.shape, text_valid.shape

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


((20453, 4623), (3610, 4623))

In [7]:
#word count and unique word counts; actually might not be so useful
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler_fit = scaler.fit(all_df[["wc","uwc"]].astype(float))
print(scaler_fit.mean_, scaler_fit.var_)
num_train = scaler_fit.transform(train_df[["wc","uwc"]].astype(float))
num_valid = scaler_fit.transform(valid_df[["wc","uwc"]].astype(float))
num_train.shape, num_valid.shape

[21.94053111 18.19224536] [1152.55429082  510.60884649]


((20453, 2), (3610, 2))

In [8]:
#concatenate text and word count features
X_train = np.concatenate([num_train,text_train.toarray()],axis=1)
X_valid = np.concatenate([num_valid,text_valid.toarray()],axis=1)
X_train.shape, X_valid.shape

((20453, 4625), (3610, 4625))

In [9]:
#fit logistic regression models
model = LogisticRegression(C=2., penalty="l2", solver="liblinear", dual=False, multi_class="ovr")
model.fit(X_train,y_train)
model.score(X_valid,y_valid)

0.7318559556786703

In [10]:
probs = model.predict_proba(X_valid)
probs_df = pd.DataFrame(probs)
probs_df.columns = model.classes_
probs_df["preds"] = model.predict(X_valid)
probs_df["category"] = valid_df.category
probs_df["texts"] = valid_df.texts
probs_df["processed"] = valid_df.processed
probs_df["wc"] = valid_df.wc
probs_df["uwc"] = valid_df.uwc
probs_df["hit"] = (probs_df.preds==probs_df.category)
probs_df.head()

,neg,neu,pos,q,preds,category,texts,processed,wc,uwc,hit
0,0.505040,0.273455,0.219629,0.001876,neg,neu,เห็นคนลบแอพ viu ก็เห็นใจและเข้าใจเขานะคะ แผลมั...,เห็น|คน|ลบ|แอ|พ|viu|ก็|เห็นใจ|และ|เข้าใจ|เขา|น...,47,42,False
1,0.101227,0.844443,0.048554,0.005776,neu,neu,ไปชมไม้คิวของแชมป์ และรองแชมป์ กันจ้า! ..........,ไป|ชม|ไม้|คิว|ของ|แชมป์|และ|รอง|แชมป์|กัน|จ้า|...,42,40,True
2,0.558792,0.349736,0.089218,0.002254,neg,neg,กลุ่มรถซีวิคเป็นกลุ่มที่น่ารำคานมากกกกกกกกก อว...,กลุ่ม|รถ|ซีวิค|เป็น|กลุ่ม|ที่|น่า|รำ|คาน|มาก|x...,46,35,True
3,0.075085,0.563439,0.356436,0.005040,neu,neu,อยากสวยเหมือนเจ้าของแบรนด์สิคะ เนย โชติกา ใบหน...,อยาก|สวย|เหมือน|เจ้าของ|แบรนด์|สิ|คะ|เนย|โชติ|...,72,56,True
4,0.727432,0.265063,0.007502,0.000003,neg,neg,ข้าวโถละร้อย แพง เพราะตักเป็นจานๆละ15 เต็มที่ก...,ข้าว|โถ|ละ|ร้อย|แพง|เพราะ|ตัก|เป็น|จาน|ๆ|ละ|15...,381,218,True


###### Create function ######

In [11]:
def assign_sentiment(pneg, pneu, ppos):
    if pneu >= 0.7:
        return 'neu'
    else:
        if max(pneg, ppos) == pneg:
            return 'neg'
        else:
            return 'pos'

In [12]:
def apply_datetime(start_time, end_time):
    x = start_time.split()[1]
    y = end_time.split()[1]
    
    
    t1 = datetime.strptime(x, "%H:%M:%S")

    t2 = datetime.strptime(y, "%H:%M:%S")

    # get difference
    delta = t2 - t1
    
    # time difference in milliseconds
    ms = delta.total_seconds() / 60
    
    return int(ms)

In [13]:
def assign_gender(x):
    male_word = ['ครับ', 'คับ', 'ผม']
    female_word = ['ค่ะ', 'คะ', 'หนู']
    for i in male_word:
        if i in x:
            return 'Male'
    for j in female_word:
        if j in x:
            return 'Female'
    return 'Undefine'

##### Prepare Data ##### 

In [14]:
import pytchat
import pandas as pd 
chat = pytchat.create(video_id="6iXz5bpLn54")

dList = []
aList = []
mList = []

i = 0
while chat.is_alive():
  for c in chat.get().sync_items():
    print(f"{c.datetime} [{c.author.name}]- {c.message}")
    dList.append(c.datetime)
    aList.append(c.author.name)
    mList.append(c.message)
    i = i+1
    if i == 1000:
      chat.terminate()
      break

# dictionary of lists 
dict = {'dateTime': dList, 'authorName': aList, 'Message': mList}

df = pd.DataFrame(dict)
df.head()

#from google.colab import files
#df.to_csv("data.csv", encoding="utf_8_sig", index=False) #cp874, #TIS-620
#files.download("data.csv")

2023-01-10 05:59:54 [Pornchai S.]- hi
2023-01-10 06:00:03 [BOY BANK]- 2
2023-01-10 06:00:04 [อภิชาต สังข์ลาย]- :person-turqouise-waving::person-turqouise-waving::person-turqouise-waving::person-turqouise-waving::person-turqouise-waving:
2023-01-10 06:00:05 [Pornchai S.]- hi
2023-01-10 06:00:20 [Dec.]- สวัสดีค่ะะะะะ
2023-01-10 06:00:23 [leo]- :face-blue-smiling:
2023-01-10 06:00:34 [Prath C]- อรุณสวัสดิ์ครับทุกคน
2023-01-10 06:00:37 [Wisanuwan Puengtumjit]- มาต่อกันครับ
2023-01-10 06:00:50 [leo]- :folded_hands:ลุงยุทธ
2023-01-10 06:01:01 [leo]- :folded_hands:ทุกคน
2023-01-10 06:01:09 [Windy blue Mc]- Good morning ค่ะ
2023-01-10 06:01:40 [alongkot khongphan]- สงสารจัง
2023-01-10 06:01:57 [alongkot khongphan]- เก่งรีบไปไหน
2023-01-10 06:02:35 [ชินจัง 1903]- ขับขี่ต้องระมัดระวัง
2023-01-10 06:02:52 [alongkot khongphan]- 8 ขวบน่าส่งสาร เพราะความประมาทของมนุษย์
2023-01-10 06:02:53 [หม่าอ๊อน]- สงสารน้องๆ
2023-01-10 06:02:57 [Sutin Nilkachatal]- เชื่อเถอะจะมีน้ำตาอาบแก้มและขอโทษสังคมคำถามง่ายๆ

2023-01-10 06:07:50 [ꪳH]- เหลือแต่รูปที่มีทุกบ้าน
2023-01-10 06:07:53 [mdfbp pd1]- ดูหน้าคนขึบ-ไม่สลดจริง
2023-01-10 06:07:56 [Mangos Repent]- :smiling_face_with_smiling_eyes::tulip:
2023-01-10 06:07:58 [Devil's Gold]- ขอให้อีป้าที่ชนลูกมรึงโดนคนชนตายไวๆ:pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face:
2023-01-10 06:07:59 [อวตาร อวตาร]- สมัยประยุทธ มีความสุขหรือ เมื่อไรจะไปสักที
2023-01-10 06:08:00 [Lisa Lisa]- ประเทศไทย ไฟไหม้บ่อยๆ นะจ๊ะ
2023-01-10 06:08:07 [Devil's Gold]- ขอให้อีป้าที่ชนลูกมรึงโดนคนชนตายไวๆ:pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::pouting_face::po

2023-01-10 06:15:01 [บัลลังก์ สุขเจริญ]- หมอ หมอ หมอ เฮ่อ
2023-01-10 06:15:01 [MOO Koingc]- ยกเว้นมีเงินหมอก็จะดูแลดี
2023-01-10 06:15:01 [ธนาภัทร🥰]- @คุณศิริพัฒน์จันทร์:hand-pink-waving::face-blue-smiling:สวัสดีครับคุณหงษ์:flamingo:
2023-01-10 06:15:09 [chit chiwchiw]- ?ญ๔
2023-01-10 06:15:11 [Nineonewan cannabis plunt psycoll]- เราชอบดาวเทียมมากกว่าดาวแท้
2023-01-10 06:15:21 [ธนาภัทร🥰]- อาร์เซน่อล:tongue::deaf_person::honey_pot::honey_pot::honey_pot::honey_pot::honey_pot::face_savoring_food:
2023-01-10 06:15:25 [mdfbp pd1]- เหมือนปกสืจ่ายแต่พาราไล่กลับ
2023-01-10 06:15:30 [สุรพร กัญญพันธ์]- เชลชีใต้:footprints:อาเซน่อล
2023-01-10 06:15:32 [Kanokparn Sriwongsa]- ไอ้ปืนใหญ่ ถล่มสบาย:smiling_face_with_hearts::smiling_face_with_hearts::smiling_face_with_hearts:
2023-01-10 06:15:41 [ต.โต้ง]- เก่งแท้ ไอ้นอล..
2023-01-10 06:15:43 [Rittidech ฤทธิเดช]- ใหญ่ก็เป็นข่าว เล็กไปก็เป็นข่าว
2023-01-10 06:15:45 [BENTLEY3 1962]- ต้องเป็นข่าว...ถึงมือหมอแน่
2023-01-10 06:15:47 [thavesak claysuwan]- สส 

2023-01-10 06:21:15 [Aor Manatsa]- 55555
2023-01-10 06:21:18 [ธนาภัทร🥰]- ยุไกล้ๆจะไล่ตบให้หมด
2023-01-10 06:21:18 [thavesak claysuwan]- ไปหาที่ศาลากลางจังหวัดเลยมีโรงหมอพร้อมทุกหน่วยงานราชการจังหวัดครบ
2023-01-10 06:21:19 [สาก-อ๊อน-อ่อน จริง'ง นะ]- ติ่ง
2023-01-10 06:21:22 [นก นก]- :face-blue-smiling::face-blue-smiling::face-blue-smiling:
2023-01-10 06:21:22 [🍁Jen Jen.hk.🍁]- :rolling_on_the_floor_laughing::rolling_on_the_floor_laughing::rolling_on_the_floor_laughing:
2023-01-10 06:21:22 [puppet]- :face-orange-raised-eyebrow::face-orange-raised-eyebrow::face-orange-raised-eyebrow::face-orange-biting-nails:
2023-01-10 06:21:28 [น้องไออุ่น กับคุณพ่ออ้วน]- อย่าชงมากเลย
2023-01-10 06:21:28 [บ้านสวนมินตรา TV]- รถไม่จำเป็นต้องขับเร็วให้เต็มสมรรถภาพของรถ ถ้ารีบก็เผื่อเวลาหน่อย ช้าหน่อยรถคันอื่นก็มีคนที่เค้ารักเหมือนกันครับ
2023-01-10 06:21:32 [บุรุษไร้นาม]- พ่อคับแม่คับ…หนูอยากมีรอยสักแบบพี่เขา
2023-01-10 06:21:37 [Rittidech ฤทธิเดช]- นี่..ก็ข่าว
2023-01-10 06:21:38 [Wisanuwan Puengtumjit]- ก็

2023-01-10 06:25:33 [สาธิต พงษ์เสนา]- แม่เตือนแล้ว...ลูกชาย ไม่ฟังเอง
2023-01-10 06:25:33 [Kkk Pj]- สวัสดีครับ
2023-01-10 06:25:33 [ณพงษกร แก้วจันทร์]- สว้สดีตอนเช้าครับพี่ยุทธ น้องไบร์ท
2023-01-10 06:25:34 [ไอ้หนุ่มรถไถ🚜🚜]- สีอะไร
2023-01-10 06:25:37 [Maria]- กะลา ต้มตุ๋นแม้แต่เด็ก
2023-01-10 06:25:42 [puppet]- เชื่อโจร ... ลงทุนกับโจร ....:face-orange-biting-nails:
2023-01-10 06:25:45 [go inno]- :face-green-smiling:มี รัฐบาล ไว้ทำไม :face-green-smiling:
2023-01-10 06:25:45 [ลูกชาวนา ชิณรินทร์]- ยุคการทุจริตเฟื่องฟูสูงสุด//กเรือรบยังล่ม/ซื้อเรือดำน้ำไม่มีเครื่องยนต์งี้...จีนเทาอีก ...งามจริงๆ ยคนี้อะไรไม่เคยเห็นก็ได้เห็นเป็นบูญก่อนตายจาก เกืดมาไม่เคยพบเคยเห็น
2023-01-10 06:25:54 [สมพร สวนพลาย]- (แฟนข่าว)อยากใด้แก้วกาแฟคะ
2023-01-10 06:25:56 [ลูกชาวนา ชิณรินทร์]- ยุคการทุจริตเฟื่องฟูสูงสุด//กเรือรบยังล่ม/ซื้อเรือดำน้ำไม่มีเครื่องยนต์งี้...จีนเทาอีก ...งามจริงๆ ยคนี้อะไรไม่เคยเห็นก็ได้เห็นเป็นบูญก่อนตายจาก เกืดมาไม่เคยพบเคยเห็น
2023-01-10 06:25:57 [thavesak claysuwan]- แบบสมัยก่อน กัมมั

...

2023-01-10 06:33:24 [OoBbMm]- เด็กบางคนตอนเด็กขวบสองขวบติดพ่อหนักเลยนะเห็นพ่อเลิกงานกลับบ้านวิ่งหาพ่อตลอด
2023-01-10 06:33:25 [Amp mie]- ตำรวจจับมิจฉาชีพได้รึยัง?
2023-01-10 06:33:30 [Prem PIANKAN]- ทำไมโดนหลอกง่ายขนาดนี้น้องเอ้ย
2023-01-10 06:33:31 [𝙊𝙫𝙚𝙧` 𝘿𝙤𝙨𝙚]- มันก็เป็นแค่กลอุบาย (ใช้การดูคลิป หรือ เปลี่ยนเป็นรูปแบบอื่น) ที่คนเสียกันเยอะ เพราะ ลงทุนครั้งแรกแล้วได้เงินจริง เลยมีความโลภ ใส่เงินเพิ่มต่อ
2023-01-10 06:33:33 [⚒️headcore war⚒️]- ตะกวดเคยจับได้เปล่าอิอิ
2023-01-10 06:33:33 [สาธิต พงษ์เสนา]- เห้อ.....มาอีกแล้ว ปั่นๆๆๆยอดวิว เงินงาม ทำได้ทุกที่
2023-01-10 06:33:37 [Hamsah Benrmad]- นายก ทำงานหน่อย
2023-01-10 06:33:37 [ชื่นใจ กนก]- มิฉาชีพเยอะมาก ดิฉันเพิ่งโดนหลอกเมื่อวันที่27ธคที่ผ่านซื้อกล่องสุ่ม
2023-01-10 06:33:37 [ปภาวิน]- ตาสว่างเถอะคนไทยเขาไม่ได้รักพวกเราหรอแต่ถ้าเราไม่รักเขาแม่งติดคุกส้นคีน
2023-01-10 06:33:39 [ธีระพงษ์ ทูลเหลือ]- ถ้ามันได้เงินง่ายขนาดนั้นผมคงไม่ต้องมานั่งหลังขดหลังแข็งทำงานงกๆหรอก
2023-01-10 06:33:40 [ทองเบิ้ม ดอร์สั้น]- รัฐบาลไม่ทำงานแล้วครับ รอรัฐบ

2023-01-10 06:36:26 [CnkL]- โปรไฟล์ปลอมถึงเปิดก็ไม่ใช่หน้าจริงคับแบบนี้
2023-01-10 06:36:27 [HengHeng 2022]- เพราะไร้การควบคุม เหยืีอโซเชียลเพิ่มขึ้น 24ชั่วโมงติดแต่มือถือ
2023-01-10 06:36:27 [Ketsanee W]- ไม่ต้องปิดค่ะ เปิดเลย อิพวกสิทธิมนุษย์ อย่ามาโลกสวยนะ
2023-01-10 06:36:28 [KR-150]- โจรนอก โจรใน
2023-01-10 06:36:28 [thavesak claysuwan]- รัฐ. ล้มเหลว คนไม่กลัวกฏหมาย
2023-01-10 06:36:31 [Ld🌵]- ผนง.:megaphone::gorilla:
2023-01-10 06:36:35 [toptop top]- ตร. ตามหลังมิจฉาชีพ ตลอด((และไม่เคยจับได้)) ((ข้อมูลรั่วไหลก็ข้าราชการเองที่ขายให้เขา)ป
2023-01-10 06:36:36 [SANTHA .CH.]- เครื่อข่ายตู้ห่าวจะสาวถึงตู่ห่าวคงยาก
2023-01-10 06:36:36 [teerapat aunwong]- โดนเหมือนกันค่ะ 6000
2023-01-10 06:36:43 [puppet]- :face-orange-frowning:ปล้นเด็ก
2023-01-10 06:36:55 [สุรพร กัญญพันธ์]- เด็กหลงเชื่อสาวสวย:face_blowing_a_kiss::face_blowing_a_kiss::face_blowing_a_kiss:
2023-01-10 06:36:59 [อัมรินทร์ ต๊ะวงศ์ษา]- โดนกันเยอะครับ.มิจฉาฉีพในมือถือแต่จับไม่ได้สักที
2023-01-10 06:37:01 [สาก-อ๊อน-อ่อน จริง'ง นะ

2023-01-10 06:39:44 [papey]- สงสารน้องครับ
2023-01-10 06:39:44 [IloYos]- คนที่คุยทาง ทรศ. มันไม่ใข่คนไทยเพราะมันพูดไม่ชัด
2023-01-10 06:39:45 [Raccoon Snitch]- เครค่ะ พ่อง
2023-01-10 06:39:46 [เพชรรัตน์ ปัญญายง]- ยังมีพวกแพคสบุ่อีก และพวกรับงานมาทำที่บ้าน
2023-01-10 06:39:47 [คฑาวุฒ วิริยะกุล]- คนที่ยุดอำนาจมาเล่นการเมืองเห็นมาหลายรายไปไม่รอดชักรายพวกริวร้อก็เลียจนหำจะเปื่อยก็หวังจะมียดถาบัลดาศักสุดท้าก็-ค-คอตกนิคนไปเชียรแค่ดูบอล-มวย-ไม่ใช้ทั้งประเทศทิดเย้อ
2023-01-10 06:39:48 [њัחŋ่∂এ∩דវເວබד จึงใช้ใจสาดแสงเพื่อนำทาง]- ชาวเน็ตช่วยกันตามเจอชัวร์
2023-01-10 06:39:49 [go inno]- :face-green-smiling:แค่ เป็น ข่าว แต่ จริงๆๆ มีกว่ากว่านี้อีกเยอะ :face-green-smiling::face-green-smiling:
2023-01-10 06:39:51 [ชินจัง 1903]- ทุนอะไรวะ 6 หมื่น
2023-01-10 06:39:52 [นาวี สีหพันธ์]- บัญชีม้า ตามสิจ้ะ
2023-01-10 06:39:53 [toptop top]- เก่งจับแต่คนคิดต่าง
2023-01-10 06:39:53 [นายแสง คําภูเวียง]- รัฐบาลมันไม่เอาจริง.ไม่ทำงานเลย
2023-01-10 06:39:53 [นาบงงี่ ตะไว้]- งานดูคลิป
2023-01-10 06:39:55 [Prairie 

2023-01-10 06:41:33 [big]- มันด่าสุดยอดแล้วครับไอ้พวกนี้
2023-01-10 06:41:33 [P. Lam]- คิดว่า ตร จะจับได้มั๊ย
2023-01-10 06:41:33 [ปภาวิน]- รายได้เสริมทีได้ตังจิงคืนล้างจานเมื่อคืนได้มา500
2023-01-10 06:41:34 [ธีระพงษ์ ทูลเหลือ]- ใจมึงมันโหดมากไอ้แอดมิน
2023-01-10 06:41:35 [ Phoenix]- คนทำบาปพวกไม่ได้ตายดีหลอก ทำเลว
2023-01-10 06:41:35 [꧁ เจ้าเมืองเข้าป่า ꧂]- จับไวๆ
2023-01-10 06:41:36 [Jarinya Dawan]- ขอให้จับมันได้ 
2023-01-10 06:41:36 [เจริญรุ่ง จุ้ยเจริญ]- ชั่วมากทำแบบนี้
2023-01-10 06:41:37 [Bird-Bird Channel]- กฏหมายประเทศไทยมันอ่อน เอื้อให้คนผิด
2023-01-10 06:41:37 [Nai KhonThai]- คนรากหญ้า หลอก คนรากหญ้า 
2023-01-10 06:41:39 [Rock fox]- พวกมิดฉาชีบ จะใช่รูปโปรจริงคง 0.0000001%มั้ง
2023-01-10 06:41:40 [Mom N Me-Poogun]- จับให้ได้นะคะได้โปรด
2023-01-10 06:41:40 [њัחŋ่∂এ∩דវເວබד จึงใช้ใจสาดแสงเพื่อนำทาง]- ใจสลายเลยครับคนเป็นพ่อแม่
2023-01-10 06:41:40 [Aristotel]- พวกที่ต้อว่าเด็ก คือน้องมันเด็กไง ทะเลาะกับเพื่อนยังผูกคอตายก็มี เด็กคือเด็ก เด็กต้องการคำแนะนำสั่งสอน
2023-01-10 06:41:

,dateTime,authorName,Message
0,2023-01-10 05:59:54,Pornchai S.,hi
1,2023-01-10 06:00:03,BOY BANK,2
2,2023-01-10 06:00:04,อภิชาต สังข์ลาย,:person-turqouise-waving::person-turqouise-wav...
3,2023-01-10 06:00:05,Pornchai S.,hi
4,2023-01-10 06:00:20,Dec.,สวัสดีค่ะะะะะ


##### Assign Sentiment ######

In [15]:
df["processed"] = df.Message.map(lambda x: "|".join(process_thai(x)))
df["wc"] = df.processed.map(lambda x: len(x.split("|")))
df["uwc"] = df.processed.map(lambda x: len(set(x.split("|"))))

In [16]:
#text faetures
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidftext = tfidf_fit.transform(df["Message"])

In [17]:
#word count and unique word counts; actually might not be so useful
from sklearn.preprocessing import StandardScaler

num_valid = scaler_fit.transform(df[["wc","uwc"]].astype(float))

data_valid = np.concatenate([num_valid,tfidftext.toarray()],axis=1)

In [18]:
x_probs = model.predict_proba(data_valid)
x_probs_df = pd.DataFrame(x_probs)
x_probs_df.columns = model.classes_
x_probs_df.head(20)

,neg,neu,pos,q
0,0.082666,0.753584,0.153079,0.010671
1,0.072436,0.740483,0.147865,0.039216
2,0.008875,0.834388,0.154073,0.002663
3,0.082666,0.753584,0.153079,0.010671
4,0.033912,0.887352,0.054793,0.023944
5,0.012179,0.833004,0.150924,0.003893
6,0.025658,0.565131,0.383940,0.025271
7,0.032979,0.777150,0.141246,0.048625
8,0.021335,0.801110,0.173738,0.003817
9,0.043666,0.725294,0.226912,0.004127


In [19]:
df['preds'] = x_probs_df.apply(lambda x: assign_sentiment(x['neg'], x['neu'], x['pos']), axis=1)

##### Assign Time of Living #####

In [28]:
start_time = (min(df['dateTime']).split())[1]
end_time = (max(df['dateTime']).split())[1]

In [29]:
from datetime import *
# convert time string to datetime
t1 = datetime.strptime(start_time, "%H:%M:%S")
print('Start time:', t1.time())

t2 = datetime.strptime(end_time, "%H:%M:%S")
print('End time:', t2.time())

# get difference
delta = t2 - t1

# time difference in seconds
print(f"Time difference is {delta.total_seconds()} seconds")

# time difference in milliseconds
ms = delta.total_seconds() / 60
print(f"Time difference is {ms} minutes")

Start time: 05:59:54
End time: 06:42:59
Time difference is 2585.0 seconds
Time difference is 43.083333333333336 minutes


In [30]:
df["mintime"] = min(df['dateTime'])
df.head()

,dateTime,authorName,Message,processed,wc,uwc,preds,time2,NewCol,Gender,mintime
0,2023-01-10 05:59:54,Pornchai S.,hi,hi,1,1,neu,2023-01-10 05:59:54,0,Undefine,2023-01-10 05:59:54
1,2023-01-10 06:00:03,BOY BANK,2,2,1,1,neu,2023-01-10 05:59:54,0,Undefine,2023-01-10 05:59:54
2,2023-01-10 06:00:04,อภิชาต สังข์ลาย,:person-turqouise-waving::person-turqouise-wav...,:|person-turqouise-waving|::|person-turqouise-...,11,3,neu,2023-01-10 05:59:54,0,Undefine,2023-01-10 05:59:54
3,2023-01-10 06:00:05,Pornchai S.,hi,hi,1,1,neu,2023-01-10 05:59:54,0,Undefine,2023-01-10 05:59:54
4,2023-01-10 06:00:20,Dec.,สวัสดีค่ะะะะะ,สวัสดี|ค่ะ|xxrep,3,3,neu,2023-01-10 05:59:54,0,Female,2023-01-10 05:59:54


In [31]:
df['timeinlive'] = df.apply(lambda x: apply_datetime(x['mintime'], x['dateTime']), axis=1)

In [32]:
df['timeinlive'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43], dtype=int64)

##### Assign Gender #####

In [33]:
df['Gender'] = df.apply(lambda x: assign_gender(x['Message']), axis=1)

##### Initial DataFrame for graph ######

In [34]:
df_bar = pd.DataFrame({'count' : df.groupby( ["preds"] ).size()}).reset_index()
df_graphbar = pd.DataFrame({'count' : df.groupby( ['timeinlive','preds'] ).size()}).reset_index()
df_graphgender = pd.DataFrame({'count' : df.groupby(['Gender']).size()}).reset_index()
df_line = df[['timeinlive', 'Gender']].pivot_table(index="timeinlive", columns="Gender",  aggfunc=len).fillna(0).astype(int).reset_index()
df_line['Total'] = df_line['Female'] + df_line['Male'] + df_line['Undefine']
df_graphpie = pd.DataFrame({'count' : df.groupby( ['preds'] ).size()}).reset_index()
df_people = pd.DataFrame({'count' : df.groupby(['authorName']).size()})
df_graphpeople = df_people['count'].sort_values(ascending=False).head(10).reset_index()

##### Dashboard by Dash #####

In [36]:
from dash import Dash, dcc, html, Input, Output, ctx
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd

sentiment = ['pos', 'neg', 'neu']

app = JupyterDash(__name__)

fig = px.line(df_line, x="timeinlive", y="Total", text="Total")
fig.update_traces(textposition='top center')

fig2 = px.bar(df_graphbar, x="preds", y="count",color="preds",color_discrete_sequence=["red", "gray","green"])

fig3 = px.bar(df_graphgender, x="Gender", y="count", color="Gender")

fig4 = px.pie(df_graphpie, values="count", names="preds", hole=.3)

fig5 = px.bar(df_graphpeople, x="authorName", y="count", text="count")

app.layout = html.Div([
    
    html.Div(children=[
        html.H1(children='Total #Commend by time'),

        html.Div(children='''
               The sentiment of sentense.
        '''),
        
        dcc.Checklist(
                  id = "checklist1",
                  options=[{'label': i, 'value': i} for i in sentiment],
                  value=['pos', 'neg', 'neu'], #Initial values,
                  inline=True),

        dcc.Graph(
            id='totalbytime-graph1',
            figure=fig, clear_on_unhover=True, clickData=None
        ),
        
        html.H1(children='%Contribution of Pos-Neu-Neg sentiment'),
        
        dcc.Graph(
            id='cont-graphpie',
            figure=fig4
        ),
        
        dcc.RangeSlider(
            min(df['timeinlive']),
            max(df['timeinlive'])+1,
            1,
            value=[min(df['timeinlive']), max(df['timeinlive'])],
            id='range-slider')
    ],style={'padding': 10, 'flex': 1}),
    
    html.Div(children=[
        html.H1(children='Total #Commend by Gender'),
        html.Div(children='''
            Click to clear filter.
        '''),
        html.Button('Clear Filter', id='btn1', n_clicks=0),
        
        dcc.Graph(
            id='gender-graph',
            figure=fig3, clickData=None
        ),
        
        html.H1(children='Top #Commend by Authorname'),
        
        dcc.Graph(
            id='topcommend-graph',
            figure=fig5
        )

    ],style={'padding': 10, 'flex': 1})
    ,
],style={'display': 'flex', 'flexDirection': 'row', 'flex-wrap': 'wrap'})

@app.callback(
    Output('totalbytime-graph1', 'figure'),
    Input('checklist1', 'value'))

def update_fig1(x):
    n_df_line = df[(df['preds'].isin(x))][['timeinlive','preds']].groupby('timeinlive').size().reset_index()
    n_df_line.columns = ['Time', 'Total']
    fig = px.line(n_df_line, x="Time", y="Total",text="Total")
    fig.update_traces(textposition='top center')
    return fig

@app.callback(
    Output('gender-graph', 'figure'),
    Input('checklist1', 'value'),
    Input('totalbytime-graph1','hoverData'),
    Input('totalbytime-graph1', 'clickData'),
    Input('btn1', 'n_clicks'))

def update_fig2(x, hover_data, click_data, n_click):
    if ctx.triggered_id == "btn1":
        n_df_graphgender = df[(df['preds'].isin(x))][['Gender','preds']].groupby('Gender').size().reset_index()
        n_df_graphgender.columns = ['Gender', 'Total']
        fig3 = px.bar(n_df_graphgender, x="Gender", y="Total", color="Gender")
        return fig3
    if hover_data is None:
        if click_data is None:
            n_df_graphgender = df[(df['preds'].isin(x))][['Gender','preds']].groupby('Gender').size().reset_index()
            n_df_graphgender.columns = ['Gender', 'Total']
            fig3 = px.bar(n_df_graphgender, x="Gender", y="Total", color="Gender")
            return fig3
        else:
            Specific_time = click_data['points'][0]['x']
            n_df_graphgender = df[(df['preds'].isin(x)) & (df['timeinlive'] == Specific_time)][['Gender','preds']].groupby('Gender').size().reset_index()
            n_df_graphgender.columns = ['Gender', 'Total']
            fig3 = px.bar(n_df_graphgender, x="Gender", y="Total", color="Gender")
            return fig3
    else:
        Specific_time = hover_data['points'][0]['x']
        n_df_graphgender = df[(df['preds'].isin(x)) & (df['timeinlive'] == Specific_time)][['Gender','preds']].groupby('Gender').size().reset_index()
        n_df_graphgender.columns = ['Gender', 'Total']
        fig3 = px.bar(n_df_graphgender, x="Gender", y="Total", color="Gender")
        return fig3

@app.callback(
    Output('cont-graphpie', 'figure'),
    Output('topcommend-graph', 'figure'),
    Input('range-slider', 'value'),
    Input('checklist1', 'value'))

def updatefig3(range_data, x):
    min_time = range_data[0]
    max_time = range_data[1]
    n_df_pie = df[(df['timeinlive'] >= min_time) & (df['timeinlive'] <= max_time)][['preds']].groupby('preds').size().reset_index()
    n_df_pie.columns = ['preds', 'count']
    fig4 = px.pie(n_df_pie, values="count", names="preds", hole=.3)
    n_df_people = pd.DataFrame({'count' : df[(df['timeinlive'] >= min_time) & (df['timeinlive'] <= max_time) & df['preds'].isin(x)].groupby(['authorName']).size()})
    n_df_graphpeople = n_df_people['count'].sort_values(ascending=False).head(10).reset_index()
    fig5 = px.bar(n_df_graphpeople, x="authorName", y="count", text="count")
    return fig4, fig5

@app.callback(
    Output('totalbytime-graph1', 'clickData'),
    Input('btn1', 'n_clicks'))
def clear_clickdata(x):
    return None


if __name__ == '__main__':
    app.run_server(debug=True, port=8054)

Dash app running on http://127.0.0.1:8054/
